In [1]:
%run ../../../ingestion/ingestion.ipynb
%run ../../../write/write.ipynb
%run ../../../alerts/webhook.ipynb

25/05/07 21:55:51 WARN Utils: Your hostname, lnb035652cps resolves to a loopback address: 127.0.1.1; using 192.168.1.172 instead (on interface wlp0s20f3)
25/05/07 21:55:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/07 21:55:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/07 21:55:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/05/07 21:55:52 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/05/07 21:55:54 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
25/05/07 21:55:54 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
# Cria uma instância da classe WebhookTeams utilizando o arquivo de configuração localizado em ../../config/teams.json
webhook = WebhookTeams("../../../config/teams.json")

In [3]:
try:
    # Cria um leitor de Parquet
    reader = ParquetReader()
    
    # Executa a leitura do arquivo Parquet e retorna um DataFrame
    df = client_code(reader, "../../../layer/prata/economia/animais_cabecas/*.parquet")
except:
    # Em caso de erro durante a leitura, envia uma notificação via Teams
    webhook.send_teams_notification("Erro de leitura!")
    pass


In [5]:
from pyspark.sql.functions import sum, round

# Calcula  a soma total de animais em cada região administrativa.
# A média é arredondada para duas casas decimais.
df_tratado = (
    df.groupBy("Data", "reg_administrativa")
      .agg(round(sum("Animais_cabecas"), 2).alias("soma_total_animais"))
      .orderBy("Data", "soma_total_animais")
)


In [6]:
# Exibe as primeiras linhas do DataFrame tratado para visualização dos dados após as transformações
df_tratado.show()

+----+--------------------+------------------+
|Data|  reg_administrativa|soma_total_animais|
+----+--------------------+------------------+
|2013|        RA de Santos|            2391.0|
|2013|      RA de Registro|          122769.0|
|2013|RA de São José do...|         1720620.0|
|2013|      RA de Barretos|         2131261.0|
|2013|    RM  de São Paulo|         3219825.0|
|2013|       RA de Itapeva|         4804060.0|
|2013|     RA de Araçatuba|         6887342.0|
|2013|         RA de Bauru|         8497107.0|
|2013|        RA de Franca|         8535866.0|
|2013|RA de Ribeirão Preto|         9245690.0|
|2013|RA de Presidente ...|       1.0026687E7|
|2013|          RA Central|        1.263734E7|
|2013|RA de São José do...|       1.8948936E7|
|2013|       RA de Marília|       5.5509489E7|
|2013|      RA de Sorocaba|       5.6338204E7|
|2013|      RA de Campinas|       8.7206145E7|
|2014|        RA de Santos|           28388.0|
|2014|      RA de Registro|          131008.0|
|2014|RA de S

In [7]:
try:
    # Cria uma instância da classe responsável por exportar em formato Parquet
    exporter = ParquetExport()

    # Exporta o DataFrame para o caminho local especificado
    exporter.export(df, "../../../layer/ouro/economia/cabecas_regiao/", destination="local")
except:
    # Em caso de erro durante a exportação, envia uma notificação via Teams
    webhook.send_teams_notification("Erro de escrita!")